# scrape the first three pages in the first group

In [21]:
from bs4 import BeautifulSoup
import requests

for i in range (1,4):
    url = 'http://www.boohee.com/food/group/1?page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    food_list = soup.find_all(class_='item clearfix')

    for food in food_list:
        food_name = food.find_all('a')[1]['title']
        calories = food.find('p').text
        URL = food.find('a')['href']
        print(food_name + calories+ '\n' + 'URL:'+'http://www.boohee.com'+ URL + '\n' + '---')
    


Easy Fun 营养粉丝(香菇炖鸡)，又叫Easy Fun 营养粉丝（香菇炖鸡味）热量：357 大卡(每100克)
URL:http://www.boohee.com/shiwu/fda3bd99
---
Easy Fun 营养粉丝(牛肉味噌味)热量：363 大卡(每100克)
URL:http://www.boohee.com/shiwu/fdbd0231
---
白粥，又叫白粥（粳米），稀饭，大米粥，白米粥，米粥，大米汤汤热量：46 大卡(每100毫升)
URL:http://www.boohee.com/shiwu/jingmizhou
---
Easy Fun 营养粉丝(番茄鸡蛋)，又叫Easy Fun 营养粉丝（番茄鸡蛋味）热量：348 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd81135c
---
Easy Fun 低脂咖喱鸡饭热量：179 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd87825c
---
Easy Fun 抹茶红豆麦片热量：381 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd1f7c6f
---
Easy Fun 高蛋白微波蛋糕预拌粉(香浓可可味)热量：389 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd42eab7
---
Easy Fun 红枣黑米圈(新包装)热量：392 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd5ad4f9
---
Easy Fun 鸡肉松烤麦片热量：455 大卡(每100克)
URL:http://www.boohee.com/shiwu/fdcdee28
---
Easy Fun 山药紫薯圈热量：391 大卡(每100克)
URL:http://www.boohee.com/shiwu/fd3ce3a1
---
稀饭，又叫白粥（籼米），大米粥，白米粥热量：59 大卡(每100毫升)
URL:http://www.boohee.com/shiwu/xianmizhou
---
鲜玉米，又叫玉米（鲜）、苞谷、珍珠米、棒子、玉蜀黍、苞米、六谷、热量：112 大卡(每100克)
URL:http

# use gavent to speed up the process 

In [63]:
from gevent import monkey
monkey.patch_all()
import gevent,requests, bs4, csv
from gevent.queue import Queue

work = Queue()

# set URLs
url_1 = 'http://www.boohee.com/food/group/{type}?page={page}'
for x in range(1, 4):
    for y in range(1, 4):
        real_url = url_1.format(type=x, page=y)
        work.put_nowait(real_url)

        
url_2 = 'http://www.boohee.com/food/view_menu?page={page}'
for x in range(1,4):
    real_url = url_2.format(page=x)
    work.put_nowait(real_url)

print(work)

<Queue queue=deque(['http://www.boohee.com/food/group/1?page=1', 'http://www.boohee.com/food/group/1?page=2', 'http://www.boohee.com/food/group/1?page=3', 'http://www.boohee.com/food/group/2?page=1', 'http://www.boohee.com/food/group/2?page=2', 'http://www.boohee.com/food/group/2?page=3', 'http://www.boohee.com/food/group/3?page=1', 'http://www.boohee.com/food/group/3?page=2', 'http://www.boohee.com/food/group/3?page=3', 'http://www.boohee.com/food/view_menu?page=1', 'http://www.boohee.com/food/view_menu?page=2', 'http://www.boohee.com/food/view_menu?page=3'])>


In [42]:
# define crawler
def crawler():
    headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
    }

    while not work.empty():
        url = work.get_nowait()
        res = requests.get(url, headers=headers)

        bs_res = bs4.BeautifulSoup(res.text, 'html.parser')

        foods = bs_res.find_all('li', class_='item clearfix')

        for food in foods:

            food_name = food.find_all('a')[1]['title']
            #提取出第2个<a>元素title属性的值，也就是食物名称。
            food_url = 'http://www.boohee.com' + food.find_all('a')[1]['href']
            #提取出第2个<a>元素href属性的值，跟'http://www.boohee.com'组合在一起，就是食物详情页的链接。
            food_calorie = food.find('p').text
            #用find在<li class="item clearfix">标签下，提取<p>元素，再用text方法留下纯文本，也提取出了食物的热量。              
            print(food_name,food_url,food_calories)
        

In [43]:
# create empty task list
tasks_list = []

# create 5 tasks  5个爬虫
for x in range(5):
    # use gavent.spawn() to execute the task 
    task = gevent.spawn(crawler)
    # append task in task list 
    tasks_list.append(task)
    
# use gavent.joinall to execute all the tasks 
gevent.joinall(tasks_list)

酸奶 http://www.boohee.com/shiwu/suannai_junzhi 热量：221 大卡(每100克)
牛奶，又叫纯牛奶、牛乳、全脂牛奶 http://www.boohee.com/shiwu/niuru_junzhi 热量：221 大卡(每100克)
无糖全脂拿铁，又叫拿铁咖啡、拿铁（全脂，无糖） http://www.boohee.com/shiwu/fdbb1893 热量：221 大卡(每100克)
奶酪，又叫乳酪、芝士、起司、计司 http://www.boohee.com/shiwu/nailao 热量：221 大卡(每100克)
酸奶(中脂) http://www.boohee.com/shiwu/suannai_zhongzhi 热量：221 大卡(每100克)
脱脂奶粉 http://www.boohee.com/shiwu/tuozhinaifen 热量：221 大卡(每100克)
酸奶(调味) http://www.boohee.com/shiwu/suannai_diaowei 热量：221 大卡(每100克)
酸奶(果料)，又叫果料酸奶 http://www.boohee.com/shiwu/suannai_guoliao 热量：221 大卡(每100克)
酸奶(果粒)，又叫果粒酸奶 http://www.boohee.com/shiwu/suannai_guoli 热量：221 大卡(每100克)
蒙牛 高钙牛奶，又叫蒙牛袋装高钙牛奶 http://www.boohee.com/shiwu/mengniugaogainiunai 热量：221 大卡(每100克)
Easy Fun 高蛋白小酥鱼(藤椒味) http://www.boohee.com/shiwu/fdc19ab3 热量：221 大卡(每100克)
鸡蛋，又叫鸡子、鸡卵、蛋 http://www.boohee.com/shiwu/jidan_junzhi 热量：221 大卡(每100克)
Easy Fun 低脂鸡胸肉肠(香辣味) http://www.boohee.com/shiwu/fd7a0fa9 热量：221 大卡(每100克)
Easy Fun 鸡胸肉丝(原味) http://www.boohee.com/shiwu/fda82c33 热量：221 

[<Greenlet at 0x1feae0077b8: _run>,
 <Greenlet at 0x1feae0076a8: _run>,
 <Greenlet at 0x1feae007488: _run>,
 <Greenlet at 0x1feae007598: _run>,
 <Greenlet at 0x1feae007268: _run>]

# save data 

In [64]:
# -------------------excel-----------------
import requests,openpyxl
wb=openpyxl.Workbook()  
sheet=wb.active 
sheet.title='food calories' 
sheet['A1'] ='name'     #加表头，给A1单元格赋值
sheet['B1'] ='calories'   #加表头，给B1单元格赋值
sheet['C1'] ='URL'   #加表头，给C1单元格赋值

def crawler():
    headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
    }
    while not work.empty():
        url = work.get_nowait()
        res = requests.get(url, headers=headers)
        bs_res = bs4.BeautifulSoup(res.text, 'html.parser')
        foods = bs_res.find_all('li', class_='item clearfix')
        for food in foods:
            food_name = food.find_all('a')[1]['title']
            food_url = 'http://www.boohee.com' + food.find_all('a')[1]['href']
            food_calorie = food.find('p').text
            sheet.append([food_name,food_calories,food_url])
            wb.save('Aug31-8.xlsx')  
            
tasks_list = []
for x in range(5):
    task = gevent.spawn(crawler)
    tasks_list.append(task)
gevent.joinall(tasks_list)
 

[<Greenlet at 0x1feafc5f9d8: _run>,
 <Greenlet at 0x1feafc5f8c8: _run>,
 <Greenlet at 0x1feafc5fae8: _run>,
 <Greenlet at 0x1feafc5fd08: _run>,
 <Greenlet at 0x1feafc5fbf8: _run>]